In [46]:
import numpy as np
import pandas as pd

# Membaca dataset
df = pd.read_csv('dataset_karyawan.csv')

# Membagi dataset menjadi data_training dan data_testing
data_training = df.iloc[:84]
data_testing = df.iloc[84:]

# Menampilkan data_training dan data_testing
print("Data Training:")
print(data_training.to_string())

print("\nData Testing:")
print(data_testing.to_string())

Data Training:
         nama                nik jenis_kelamin lama_bekerja kualitas_kerja     perilaku          absensi naik_jabatan
0   BIG-00069     Syarif Hidayat     Laki-laki       Senior           Baik         Baik  Kurang Disiplin           Ya
1   BIG-00068    Rina Susilowati     Perempuan       Senior           Baik  Kurang Baik         Disiplin           Ya
2   BIG-00005       Eko Prasetyo     Laki-laki       Senior           Baik         Baik  Kurang Disiplin           Ya
3   BIG-00006       Fani Permata     Perempuan       Senior           Baik         Baik  Kurang Disiplin           Ya
4   BIG-00008       Hari Nugroho     Laki-laki       Senior           Baik         Baik  Kurang Disiplin           Ya
5   BIG-00010      Joko Sutrisno     Laki-laki       Senior           Baik  Kurang Baik         Disiplin           Ya
6   BIG-00014        Nia Pratiwi     Perempuan       Senior           Baik         Baik         Disiplin           Ya
7   BIG-00017         Qori Rahma     Pere

In [47]:
# Menghitung jumlah baris dan kolom data training dan testing
for name, data in [("Training", data_training), ("Testing", data_testing)]:
    print(f"Jumlah Baris & Kolom Data {name}")
    print(f"Jumlah baris: {len(data)}")
    print(f"Jumlah kolom: {data.shape[1]}\n")

# Mengecek apakah ada sel yang kosong
print("\nCek data kosong:")
if df.isnull().values.any():
    print("Terdapat data yang kosong!")
    print(df.isnull().sum())  # Menampilkan jumlah data kosong per kolom
else:
    print("Tidak ada data kosong.")

# Mengecek apakah ada data duplikat
print("\nCek data duplikat:")
if df.duplicated().any():
    print("Terdapat data duplikat!")
    print(f"Jumlah data duplikat: {df.duplicated().sum()}")
else:
    print("Tidak ada data duplikat.")

Jumlah Baris & Kolom Data Training
Jumlah baris: 84
Jumlah kolom: 8

Jumlah Baris & Kolom Data Testing
Jumlah baris: 24
Jumlah kolom: 8


Cek data kosong:
Tidak ada data kosong.

Cek data duplikat:
Tidak ada data duplikat.


In [58]:
# Menentukan kolom fitur dan kolom target
target_column = "naik_jabatan"
feature_columns = ["lama_bekerja", "kualitas_kerja", "perilaku", "absensi"]

# Menghitung probabilitas total untuk kelas target
total_data = len(data_training)
prob_yes_total = len(data_training[data_training[target_column] == "Ya"]) / total_data
prob_no_total = len(data_training[data_training[target_column] == "Tidak"]) / total_data

print(f"\nProbabilitas Total:")
print(f"P(Ya) = {prob_yes_total:.4f}")
print(f"P(Tidak) = {prob_no_total:.4f}")

# Memisahkan data berdasarkan kelas target
data_ya = data_training[data_training[target_column] == "Ya"]
data_tidak = data_training[data_training[target_column] == "Tidak"]

# Menghitung probabilitas untuk setiap fitur
probabilities = {}

for feature in feature_columns:
    unique_values = data_training[feature].unique()  # Nilai unik pada fitur
    for value in unique_values:
        # P(fitur=value | Ya)
        prob_yes = len(data_ya[data_ya[feature] == value]) / len(data_ya)
        # P(fitur=value | Tidak)
        prob_no = len(data_tidak[data_tidak[feature] == value]) / len(data_tidak)

        # Menyimpan probabilitas dalam dictionary
        probabilities[f"P({feature}={value}|Ya)"] = prob_yes
        probabilities[f"P({feature}={value}|Tidak)"] = prob_no

# Menampilkan hasil probabilitas
print("\nHasil Probabilitas Tiap Fitur:")
for key, value in probabilities.items():
    print(f"{key} = {value:.4f}")



Probabilitas Total:
P(Ya) = 0.6905
P(Tidak) = 0.3095

Hasil Probabilitas Tiap Fitur:
P(lama_bekerja=Senior|Ya) = 0.7586
P(lama_bekerja=Senior|Tidak) = 0.7692
P(lama_bekerja=Junior|Ya) = 0.2414
P(lama_bekerja=Junior|Tidak) = 0.2308
P(kualitas_kerja=Baik|Ya) = 0.8966
P(kualitas_kerja=Baik|Tidak) = 0.2692
P(kualitas_kerja=Kurang Baik|Ya) = 0.1034
P(kualitas_kerja=Kurang Baik|Tidak) = 0.7308
P(perilaku=Baik|Ya) = 0.9483
P(perilaku=Baik|Tidak) = 0.4231
P(perilaku=Kurang Baik|Ya) = 0.0517
P(perilaku=Kurang Baik|Tidak) = 0.5769
P(absensi=Kurang Disiplin|Ya) = 0.3103
P(absensi=Kurang Disiplin|Tidak) = 0.6154
P(absensi=Disiplin|Ya) = 0.6897
P(absensi=Disiplin|Tidak) = 0.3846


In [57]:
# Prediksi menggunakan Naive Bayes pada data_testing
def predict_naive_bayes(row):
    prob_yes = prob_yes_total
    prob_no = prob_no_total

    for feature in feature_columns:
        value = row[feature]
        prob_yes *= probabilities.get(f"P({feature}={value}|Ya)", 0)
        prob_no *= probabilities.get(f"P({feature}={value}|Tidak)", 0)

    return "Ya" if prob_yes > prob_no else "Tidak"

data_testing.loc[:, "prediksi"] = data_testing.apply(predict_naive_bayes, axis=1)
data_testing.loc[:, "akurasi"] = data_testing["prediksi"] == data_testing[target_column]
data_testing.loc[:, "hasil"] = data_testing["akurasi"].apply(lambda x: "Akurat" if x else "Tidak Akurat")

# Menampilkan hasil
print("\nHasil Prediksi dan Akurasi:")
print(data_testing[["nama", "nik", "jenis_kelamin", "lama_bekerja", "kualitas_kerja", "perilaku", "absensi", target_column, "prediksi", "hasil"]].to_string(index=False))



Hasil Prediksi dan Akurasi:
     nama              nik jenis_kelamin lama_bekerja kualitas_kerja    perilaku         absensi naik_jabatan prediksi        hasil
BIG-00085     Juwita Putri     Perempuan       Junior           Baik        Baik        Disiplin           Ya       Ya       Akurat
BIG-00086        Kunto Adi     Laki-laki       Junior           Baik        Baik        Disiplin           Ya       Ya       Akurat
BIG-00087 Lilis Setiyowati     Perempuan       Junior           Baik        Baik        Disiplin           Ya       Ya       Akurat
BIG-00088    Maman Suryadi     Laki-laki       Junior           Baik        Baik        Disiplin           Ya       Ya       Akurat
BIG-00089     Nina Hartati     Perempuan       Senior           Baik        Baik        Disiplin           Ya       Ya       Akurat
BIG-00090    Oni Supriyadi     Laki-laki       Senior           Baik        Baik        Disiplin           Ya       Ya       Akurat
BIG-00091    Puput Lestari     Perempuan       